In [1]:
import pandas as pd

# pip install dictdiffer
import dictdiffer 


import warnings
warnings.filterwarnings('ignore')


import h2o
h2o.init(min_mem_size='25G')



Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_265"; OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~16.04-b01); OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
  Starting server from /anaconda/envs/azureml_py36/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprrfqmwi5
  JVM stdout: /tmp/tmprrfqmwi5/h2o_azureuser_started_from_python.out
  JVM stderr: /tmp/tmprrfqmwi5/h2o_azureuser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,13 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,5 months !!!
H2O_cluster_name:,H2O_from_python_azureuser_a0j2bu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,23.96 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
def get_models_from_model_ids(model_ids):
    models = []
    for id in model_ids:
        models.append(h2o.get_model(id))
    
    return models


In [5]:
train = h2o.import_file("../data/processed/final.train.tsv")
test = h2o.import_file("../data/processed/final.test.tsv")


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
# Identify predictors and response
train_predictor_cols = train.columns
train_response_col = "Resistance_Status"
train_predictor_cols.remove('SampleID')
train_predictor_cols.remove(train_response_col)
print("train frame - predictor column: ", train_predictor_cols[0], train_predictor_cols[-1])
print("train frame - response column: ", train_response_col)



# Identify predictors and response
test_predictor_cols = test.columns
test_response_col = "Resistance_Status"
test_predictor_cols.remove('SampleID')
test_predictor_cols.remove(test_response_col)
print("test frame - predictor columns: ", test_predictor_cols[0], test_predictor_cols[-1])
print("test frame - response column: ", test_response_col)


# For binary classification, response should be a factor
train[train_response_col] = train[train_response_col].asfactor()
test[test_response_col] = test[test_response_col].asfactor()


# Number of CV folds (to generate level-one data for stacking)
nfolds = 5


x = train_predictor_cols
y = train_response_col



train frame - predictor column:  NC000962_3.22 NC000962_3.4411327
train frame - response column:  Resistance_Status
test frame - predictor columns:  NC000962_3.22 NC000962_3.4411327
test frame - response column:  Resistance_Status


## Load saved models

### Naive-Bayes

In [10]:
nb_grid = h2o.load_grid("../models/ALL_FEATURES/nb_grid/Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205")
nb_grid

     compute_metrics eps_prob eps_sdev laplace min_prob min_sdev  \
0               true      1.0      1.0     1.0      0.3      0.6   
1               true      1.0      1.0     0.9      0.1      0.6   
2               true      0.9      0.9     0.1      0.1      1.0   
3              false      0.1      0.9     0.3      0.9      1.0   
4               true      0.3      1.0     0.1      0.3      0.9   
5               true      0.3      0.6     0.3      1.0      0.6   
6               true      1.0      0.3     1.0      0.6      0.6   
7               true      0.1      1.0     1.0      0.1      0.1   
8               true      0.1      0.9     0.6      0.9      0.6   
9               true      0.6      0.3     0.1      0.9      1.0   
10              true      1.0      1.0     0.1      0.1      0.3   
11             false      0.9      0.9     0.1      1.0      0.1   
12              true      0.6      1.0     0.6      0.3      0.3   
13              true      0.3      0.9     0.1  

In [12]:
nb_grid.predict(test)

naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progre

predict,p0,p1
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878
1,0.346122,0.653878


predict,p0,p1
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903
1,0.346097,0.653903


predict,p0,p1
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104
1,0.345896,0.654104


predict,p0,p1
0,0.999315,0.000685367
0,1,3.48997e-07
1,0.0081186,0.991881
1,1.54887e-07,1
1,2.20052e-06,0.999998
1,6.89905e-08,1
1,2.11575e-08,1
0,0.984356,0.0156439
0,0.966558,0.033442
0,0.971588,0.028412


...

predict,p0,p1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1


predict,p0,p1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1
1,0,1


predict,p0,p1
1,1.09781e-07,1
0,0.999994,6.02523e-06
1,1.62889e-13,1
0,1,2.05183e-21
0,1,1.81821e-18
0,1,1.11875e-18
0,1,1.23327e-20
1,3.73903e-06,0.999996
1,9.01203e-12,1
1,0.422562,0.577438


predict,p0,p1
1,9.50766e-14,1
1,2.86504e-27,1
1,2.64886e-16,1
1,4.40817e-43,1
1,1.19743e-33,1
1,6.11516e-40,1
1,6.05194e-43,1
1,4.48264e-20,1
1,5.85767e-17,1
1,4.30129e-27,1


{'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_5': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_13': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_12': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_11': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_18': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_14': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_17': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_9': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_10': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_19': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_16': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_2': ,
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_8': ,
 'G

In [13]:
sorted_grid = nb_grid.get_grid(sort_by='auc', decreasing=True)
sorted_grid

     compute_metrics eps_prob eps_sdev laplace min_prob min_sdev  \
0               true      0.1      0.9     0.6      0.9      0.6   
1              false      0.1      0.9     0.3      0.9      1.0   
2               true      0.1      1.0     1.0      0.1      0.1   
3               true      0.3      1.0     1.0      0.3      0.3   
4               true      0.3      1.0     0.1      0.3      0.9   
5               true      0.6      1.0     0.6      0.3      0.3   
6              false      0.6      0.6     0.9      1.0      0.1   
7              false      0.9      0.9     0.1      1.0      0.1   
8               true      0.3      0.9     0.1      0.6      0.1   
9               true      0.3      0.6     0.3      1.0      0.6   
10              true      1.0      1.0     0.1      0.1      0.3   
11             false      0.6      0.3     0.9      1.0      0.1   
12              true      0.3      0.1     0.9      0.6      1.0   
13              true      0.9      0.1     1.0  

In [19]:
top_5_nb_model_ids = sorted_grid.model_ids[:5]
top_5_nb_model_ids

['Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_10',
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_11',
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_9',
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_3',
 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_18']

In [28]:
model = h2o.get_model('Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_10')
model.auc()

0.7288534278959811

In [35]:
top_5_nb_model_id_params_dict = {}

for mdl_id in top_5_nb_model_ids:
    model = h2o.get_model(mdl_id)
    top_5_nb_model_id_params_dict[mdl_id] = {'params': model.actual_params, 
                                             'auc': model.auc(), 
                                            }


top_5_nb_model_id_params_dict

{'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_10': {'params': {'model_id': 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_10',
   'nfolds': 5,
   'seed': 1234,
   'fold_assignment': 'Random',
   'fold_column': None,
   'keep_cross_validation_models': True,
   'keep_cross_validation_predictions': True,
   'keep_cross_validation_fold_assignment': False,
   'training_frame': None,
   'validation_frame': None,
   'response_column': 'Resistance_Status',
   'ignored_columns': ['SampleID'],
   'ignore_const_cols': True,
   'score_each_iteration': False,
   'balance_classes': False,
   'class_sampling_factors': None,
   'max_after_balance_size': 5.0,
   'max_confusion_matrix_size': 20,
   'max_hit_ratio_k': 0,
   'laplace': 0.6,
   'min_sdev': 0.6,
   'eps_sdev': 0.9,
   'min_prob': 0.9,
   'eps_prob': 0.1,
   'compute_metrics': True,
   'max_runtime_secs': 1.7976931348623157e+308,
   'export_checkpoints_dir': None},
  'auc': 0.7288534278959811},
 

In [36]:
import json

with open('../models/ALL_FEATURES/top_5_nb_models.json', 'w') as json_file:
  json.dump(top_5_nb_model_id_params_dict, json_file)

### GLM

In [37]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OGeneralizedLinearEstimator



hyper_params = { # lambda //use self.lambda_
"alpha" : [0, 0.3, 0.6, 0.9, 1],
# "missing_values_handling" : ["mean_imputation", "skip", "plug_values"],
"theta" : [0, 0.3, 0.6, 0.9, 1],
"tweedie_link_power" : [0, 0.3, 0.6, 0.9, 1, 3, 6, 9],
"tweedie_variance_power" : [0, 0.3, 0.6, 0.9, 1, 3, 6, 9],
}

search_criteria = {"strategy": "RandomDiscrete", 
                   "max_models": 20}



# Train and cross-validate a NB
base_model = H2OGeneralizedLinearEstimator(
                                family= "binomial",

                                 nfolds=nfolds, 
                                 fold_assignment = "random",
                                 keep_cross_validation_predictions = True,
                                 seed=1234)




# Train the grid
glm_grid = H2OGridSearch(model=base_model,
                     hyper_params=hyper_params,
                     search_criteria=search_criteria)


glm_grid.train(x=x, y=y, training_frame=train, validation_frame=test) 

KeyboardInterrupt: 

### GBM

In [42]:
GBM_1_AutoML_20201101_101936_auc93 
GBM_4_AutoML_20201101_101936_auc92 
GBM_2_AutoML_20201101_101936_auc92 
GBM_5_AutoML_20201101_101936_auc85 
GBM_3_AutoML_20201101_101936_auc93 




In [43]:
model.summary

Model Details
H2ONaiveBayesEstimator :  Naive Bayes
Model Key:  Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_2


Model Summary: 


ModelMetricsBinomial: naivebayes
** Reported on validation data. **

MSE: 0.6230624103247986
RMSE: 0.7893430244987274
LogLoss: 9.439694348399003
Mean Per-Class Error: 0.3328674628816205


,,number_of_response_levels,min_apriori_probability,max_apriori_probability
0,,2.0,0.345896,0.654104


AUC: 0.6726413039532435
AUCPR: 0.7764300479188593
Gini: 0.345282607906487

Confusion Matrix (Act/Pred) for max f1 @ threshold = 8.378671271492799e-13: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score:  7.75 %


ModelMetricsBinomial: naivebayes
** Reported on cross-validation data. **

MSE: 0.49401999576322275
RMSE: 0.7028655602341196
LogLoss: 10.022553621849632
Mean Per-Class Error: 0.35708776595744685
AUC: 0.6625458037825058
AUCPR: 0.7971565850654403
Gini: 0.32509160756501165

Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.2727881169610487e-10: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 65.41 %, avg score: 23.65 %


Cross-Validation Metrics Summary: 

See the whole table with table.as_data_frame()


,,0,1,Error,Rate
0,0,0.0,163.0,1.0,(163.0/163.0)
1,1,0.0,338.0,0.0,(0.0/338.0)
2,Total,0.0,501.0,0.3253,(163.0/501.0)


,metric,threshold,value,idx
0,max f1,8.378671e-13,0.805721,399.0
1,max f2,8.378671e-13,0.912035,399.0
2,max f0point5,3.835688e-07,0.765070,223.0
3,max accuracy,6.048750e-10,0.686627,389.0
4,max precision,1.108116e-05,0.863636,116.0
5,max recall,8.378671e-13,1.000000,399.0
6,max specificity,1.000000e+00,0.957055,0.0
7,max absolute_mcc,3.835688e-07,0.313540,223.0
8,max min_per_class_accuracy,1.381328e-07,0.644172,260.0
9,max mean_per_class_accuracy,3.835688e-07,0.667133,223.0


...

,,0,1,Error,Rate
0,0,0.0,423.0,1.0,(423.0/423.0)
1,1,0.0,800.0,0.0,(0.0/800.0)
2,Total,0.0,1223.0,0.3459,(423.0/1223.0)


,metric,threshold,value,idx
0,max f1,1.272788e-10,0.790905,399.0
1,max f2,1.272788e-10,0.904364,399.0
2,max f0point5,1.101170e-08,0.741124,398.0
3,max accuracy,1.272788e-10,0.654129,399.0
4,max precision,9.999998e-01,0.966102,4.0
5,max recall,1.272788e-10,1.000000,399.0
6,max specificity,1.000000e+00,0.990544,0.0
7,max absolute_mcc,1.101170e-08,0.272315,398.0
8,max min_per_class_accuracy,1.101170e-08,0.626250,398.0
9,max mean_per_class_accuracy,1.101170e-08,0.642912,398.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.020442,1.000000e+00,1.467600,1.467600,0.960000,1.000000e+00,0.960000,1.000000,0.03000,0.03000,46.760000,46.760000
1,,2,0.020442,1.000000e+00,0.000000,1.467600,0.000000,0.000000e+00,0.960000,1.000000,0.00000,0.03000,-100.000000,46.760000
2,,3,0.030253,1.000000e+00,1.528750,1.487432,1.000000,1.000000e+00,0.972973,1.000000,0.01500,0.04500,52.875000,48.743243
3,,4,0.040065,1.000000e+00,1.401354,1.466352,0.916667,1.000000e+00,0.959184,1.000000,0.01375,0.05875,40.135417,46.635204
4,,5,0.050695,1.000000e+00,1.411154,1.454778,0.923077,1.000000e+00,0.951613,1.000000,0.01500,0.07375,41.115385,45.477823
5,,6,0.100572,9.999997e-01,1.478627,1.466606,0.967213,9.999999e-01,0.959350,1.000000,0.07375,0.14750,47.862705,46.660569
6,,7,0.150450,9.999582e-01,1.102705,1.345965,0.721311,9.999888e-01,0.880435,0.999996,0.05500,0.20250,10.270492,34.596467
7,,8,0.200327,9.990533e-01,1.228012,1.316597,0.803279,9.998026e-01,0.861224,0.999948,0.06125,0.26375,22.801230,31.659694
8,,9,0.300082,9.298661e-04,1.027520,1.220501,0.672131,3.623897e-01,0.798365,0.788008,0.10250,0.36625,2.752049,22.050068
9,,10,0.399836,4.129111e-06,1.140297,1.200491,0.745902,1.078321e-04,0.785276,0.591436,0.11375,0.48000,14.029713,20.049080


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.74085116,0.024191575,0.7389558,0.734127,0.7447699,0.7095436,0.7768595
1,auc,0.77306044,0.021443421,0.77611834,0.79549605,0.7602201,0.74347824,0.7899896
2,aucpr,0.85476434,0.0061987354,0.85713553,0.85604465,0.8438958,0.85730803,0.85943764
3,err,0.25914884,0.024191575,0.26104417,0.26587301,0.25523013,0.29045644,0.2231405
4,err_count,63.4,6.1886992,65.0,67.0,61.0,70.0,54.0
5,f0point5,0.77596784,0.03441337,0.7824223,0.754717,0.7696127,0.7419355,0.83115184
6,f1,0.82380354,0.007052221,0.8179272,0.819407,0.8355795,0.8214286,0.8246753
7,f2,0.8810496,0.042874616,0.85680753,0.8962264,0.9139151,0.92,0.818299
8,lift_top_group,1.4284575,0.23887382,1.0060606,1.5849056,1.5031446,1.4968944,1.551282
9,logloss,10.032916,6.0386367,7.355562,12.506874,18.742535,2.575395,8.984215


<bound method ModelBase.summary of >

In [44]:
model.actual_params

{'model_id': 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_2',
 'nfolds': 5,
 'seed': 1234,
 'fold_assignment': 'Random',
 'fold_column': None,
 'keep_cross_validation_models': True,
 'keep_cross_validation_predictions': True,
 'keep_cross_validation_fold_assignment': False,
 'training_frame': None,
 'validation_frame': None,
 'response_column': 'Resistance_Status',
 'ignored_columns': ['SampleID'],
 'ignore_const_cols': True,
 'score_each_iteration': False,
 'balance_classes': False,
 'class_sampling_factors': None,
 'max_after_balance_size': 5.0,
 'max_confusion_matrix_size': 20,
 'max_hit_ratio_k': 0,
 'laplace': 0.1,
 'min_sdev': 0.1,
 'eps_sdev': 0.9,
 'min_prob': 1.0,
 'eps_prob': 0.9,
 'compute_metrics': False,
 'max_runtime_secs': 1.7976931348623157e+308,
 'export_checkpoints_dir': None}

In [14]:
model1 = h2o.load_model("../models/ALL_FEATURES/nb_grid/Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_1")

In [46]:
model1.actual_params

dict

In [50]:
a_dict= model.actual_params
b_dict= model1.actual_params

for diff in list(dictdiffer.diff(a_dict, b_dict)):         
    print(diff)

('change', 'model_id', ('Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_2', 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_1'))
('change', 'laplace', (0.1, 0.9))
('change', 'eps_sdev', (0.9, 0.6))
('change', 'eps_prob', (0.9, 0.6))


## Train similar algorithm with the same parameters

In [66]:
model1.actual_params

{'model_id': 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_1',
 'nfolds': 5,
 'seed': 1234,
 'fold_assignment': 'Random',
 'fold_column': None,
 'keep_cross_validation_models': True,
 'keep_cross_validation_predictions': True,
 'keep_cross_validation_fold_assignment': False,
 'training_frame': None,
 'validation_frame': None,
 'response_column': 'Resistance_Status',
 'ignored_columns': ['SampleID'],
 'ignore_const_cols': True,
 'score_each_iteration': False,
 'balance_classes': False,
 'class_sampling_factors': None,
 'max_after_balance_size': 5.0,
 'max_confusion_matrix_size': 20,
 'max_hit_ratio_k': 0,
 'laplace': 0.9,
 'min_sdev': 0.1,
 'eps_sdev': 0.6,
 'min_prob': 1.0,
 'eps_prob': 0.6,
 'compute_metrics': False,
 'max_runtime_secs': 1.7976931348623157e+308,
 'export_checkpoints_dir': None}

In [68]:
{**model1.actual_params}

{'model_id': 'Grid_NaiveBayes_py_5_sid_8e0c_model_python_1604225741410_1205_model_1',
 'nfolds': 5,
 'seed': 1234,
 'fold_assignment': 'Random',
 'fold_column': None,
 'keep_cross_validation_models': True,
 'keep_cross_validation_predictions': True,
 'keep_cross_validation_fold_assignment': False,
 'training_frame': 10,
 'validation_frame': None,
 'response_column': 'Resistance_Status',
 'ignored_columns': ['SampleID'],
 'ignore_const_cols': True,
 'score_each_iteration': False,
 'balance_classes': False,
 'class_sampling_factors': None,
 'max_after_balance_size': 5.0,
 'max_confusion_matrix_size': 20,
 'max_hit_ratio_k': 0,
 'laplace': 0.9,
 'min_sdev': 0.1,
 'eps_sdev': 0.6,
 'min_prob': 1.0,
 'eps_prob': 0.6,
 'compute_metrics': False,
 'max_runtime_secs': 1.7976931348623157e+308,
 'export_checkpoints_dir': None}

In [73]:
mdl = H2ONaiveBayesEstimator(

 nfolds= 5,
 seed= 1234,
 fold_assignment= 'Random',
 fold_column= None,
 keep_cross_validation_models= True,
 keep_cross_validation_predictions= True,
 keep_cross_validation_fold_assignment= False,
 ignore_const_cols= True,
 score_each_iteration= False,
 balance_classes= False,
 class_sampling_factors= None,
 max_after_balance_size= 5.0,
 max_confusion_matrix_size= 20,
 max_hit_ratio_k= 0,
 laplace= 0.9,
 min_sdev= 0.1,
 eps_sdev= 0.6,
 min_prob= 1.0,
 eps_prob= 0.6,
 compute_metrics= False,
 export_checkpoints_dir= None)


mdl.train(x=x, y=y, training_frame=train, validation_frame=test) 

mdl


naivebayes Model Build progress: |████████████████████████████████████████| 100%
Model Details
H2ONaiveBayesEstimator :  Naive Bayes
Model Key:  NaiveBayes_model_python_1604225741410_5596


Model Summary: 


,,number_of_response_levels,min_apriori_probability,max_apriori_probability
0,,2.0,0.346097,0.653903




ModelMetricsBinomial: naivebayes
** Reported on validation data. **

MSE: 0.6088986395173629
RMSE: 0.7803195752493737
LogLoss: 9.264358009127474
Mean Per-Class Error: 0.3510999382872908
AUC: 0.658338476059099
AUCPR: 0.7693302572383104
Gini: 0.316676952118198

Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.6220420045776861e-12: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score:  9.21 %


ModelMetricsBinomial: naivebayes
** Reported on cross-validation data. **

MSE: 0.4976600357752729
RMSE: 0.7054502362146269
LogLoss: 10.91944247185881
Mean Per-Class Error: 0.34568853427895985
AUC: 0.6664420803782505
AUCPR: 0.797701018048522
Gini: 0.33288416075650096

Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.484107825917021e-11: 

Maximum Metrics: Maximum metrics at their respective thresholds


,,0,1,Error,Rate
0,0,0.0,163.0,1.0,(163.0/163.0)
1,1,0.0,338.0,0.0,(0.0/338.0)
2,Total,0.0,501.0,0.3253,(163.0/501.0)


,metric,threshold,value,idx
0,max f1,1.622042e-12,0.805721,399.0
1,max f2,1.622042e-12,0.912035,399.0
2,max f0point5,1.191813e-07,0.761745,270.0
3,max accuracy,4.758154e-10,0.680639,392.0
4,max precision,1.864487e-05,0.863636,115.0
5,max recall,1.622042e-12,1.000000,399.0
6,max specificity,1.000000e+00,0.957055,0.0
7,max absolute_mcc,1.191813e-07,0.281809,270.0
8,max min_per_class_accuracy,1.654924e-07,0.639053,257.0
9,max mean_per_class_accuracy,1.303375e-07,0.648900,265.0


...

,,0,1,Error,Rate
0,0,0.0,423.0,1.0,(423.0/423.0)
1,1,0.0,800.0,0.0,(0.0/800.0)
2,Total,0.0,1223.0,0.3459,(423.0/1223.0)


,metric,threshold,value,idx
0,max f1,1.484108e-11,0.790905,399.0
1,max f2,1.484108e-11,0.904364,399.0
2,max f0point5,3.601569e-10,0.750297,398.0
3,max accuracy,1.484108e-11,0.654129,399.0
4,max precision,9.999997e-01,0.961905,4.0
5,max recall,1.484108e-11,1.000000,399.0
6,max specificity,1.000000e+00,0.990544,0.0
7,max absolute_mcc,3.601569e-10,0.293984,398.0
8,max min_per_class_accuracy,3.601569e-10,0.632500,398.0
9,max mean_per_class_accuracy,3.601569e-10,0.654311,398.0



Gains/Lift Table: Avg response rate: 65.41 %, avg score: 22.07 %


Cross-Validation Metrics Summary: 

See the whole table with table.as_data_frame()


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.018806,1.000000e+00,1.528750,1.528750,1.000000,1.000000e+00,1.000000,1.000000,0.02875,0.02875,52.875000,52.875000
1,,2,0.022077,1.000000e+00,1.146562,1.472130,0.750000,1.000000e+00,0.962963,1.000000,0.00375,0.03250,14.656250,47.212963
2,,3,0.030253,1.000000e+00,1.528750,1.487432,1.000000,1.000000e+00,0.972973,1.000000,0.01250,0.04500,52.875000,48.743243
3,,4,0.040065,1.000000e+00,1.401354,1.466352,0.916667,1.000000e+00,0.959184,1.000000,0.01375,0.05875,40.135417,46.635204
4,,5,0.050695,1.000000e+00,1.411154,1.454778,0.923077,1.000000e+00,0.951613,1.000000,0.01500,0.07375,41.115385,45.477823
5,,6,0.100572,9.999908e-01,1.453566,1.454177,0.950820,9.999989e-01,0.951220,0.999999,0.07250,0.14625,45.356557,45.417683
6,,7,0.150450,9.942224e-01,1.253074,1.387507,0.819672,9.988436e-01,0.907609,0.999616,0.06250,0.20875,25.307377,38.750679
7,,8,0.200327,9.183396e-01,1.077643,1.310357,0.704918,9.694092e-01,0.857143,0.992095,0.05375,0.26250,7.764344,31.035714
8,,9,0.300082,2.951471e-05,1.027520,1.216335,0.672131,2.199444e-01,0.795640,0.735413,0.10250,0.36500,2.752049,21.633515
9,,10,0.399836,9.812856e-08,1.127766,1.194238,0.737705,3.653452e-06,0.781186,0.551937,0.11250,0.47750,12.776639,19.423824


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.7432512,0.024023822,0.74698794,0.7380952,0.7447699,0.7095436,0.7768595
1,auc,0.76616967,0.02633042,0.76608944,0.7945493,0.75998425,0.726087,0.7841383
2,aucpr,0.8487915,0.0043183267,0.84857494,0.8534926,0.8446811,0.8443926,0.8528162
3,err,0.25674877,0.024023822,0.25301206,0.26190478,0.25523013,0.29045644,0.2231405
4,err_count,62.8,5.974948,63.0,66.0,61.0,70.0,54.0
5,f0point5,0.77597517,0.033293646,0.78778136,0.7536232,0.7696127,0.7419355,0.8269231
6,f1,0.82657146,0.0054394808,0.8235294,0.82539684,0.8355795,0.8214286,0.8269231
7,f2,0.887159,0.040783882,0.8626761,0.9122807,0.9139151,0.92,0.8269231
8,lift_top_group,1.4686999,0.15053996,1.2072728,1.5849056,1.5031446,1.4968944,1.551282
9,logloss,10.923886,6.052298,9.333387,13.057011,19.005009,2.3136234,10.910403


## Train a grid of models

In [53]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ONaiveBayesEstimator

MAX_MODELS= 10


hyper_params = {
                "laplace": [0.1, 0.6, 0.9],
                "eps_sdev": [0.6, 0.9],
                "eps_prob": [0.6, 0.9],
                "compute_metrics": [True, False]
                }

search_criteria = {"strategy": "RandomDiscrete", 
                   "max_models": MAX_MODELS}



base_model = H2ONaiveBayesEstimator(
                                 nfolds=nfolds, 
                                 fold_assignment = "random",
                                 keep_cross_validation_predictions = True,
                                 seed=1234)




# Train the grid
nb_grid = H2OGridSearch(model=base_model,
                     hyper_params=hyper_params,
                     search_criteria=search_criteria)


nb_grid.train(x=x, y=y, training_frame=train, validation_frame=test) 



naivebayes Grid Build progress: |█████████████████████████████████████████| 100%


In [58]:
nb_grid_sorted = nb_grid.get_grid(sort_by='auc', decreasing=True)
nb_grid_sorted

    compute_metrics eps_prob eps_sdev laplace  \
0              true      0.6      0.6     0.1   
1             false      0.6      0.6     0.1   
2              true      0.9      0.6     0.6   
3             false      0.6      0.6     0.9   
4             false      0.9      0.6     0.6   
5              true      0.9      0.6     0.1   
6             false      0.6      0.9     0.9   
7              true      0.6      0.9     0.1   
8              true      0.9      0.9     0.6   
9             false      0.6      0.6     0.6   

                                                               model_ids  \
0  Grid_NaiveBayes_py_1_sid_82fb_model_python_1604225741410_5368_model_3   
1  Grid_NaiveBayes_py_1_sid_82fb_model_python_1604225741410_5368_model_4   
2  Grid_NaiveBayes_py_1_sid_82fb_model_python_1604225741410_5447_model_1   
3  Grid_NaiveBayes_py_1_sid_82fb_model_python_1604225741410_5447_mode...   
4  Grid_NaiveBayes_py_1_sid_82fb_model_python_1604225741410_5447_model_2   
5  G

## Stacked ensembles with different meta-learners

In [62]:
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

top_n_models = 3
collection_of_models = get_models_from_model_ids(nb_grid_sorted.model_ids[:top_n_models])


metalearner_algorithm = ["auto", "xgboost", "drf", "gbm", "glm", "naivebayes", "deeplearning"]

ensemble_list = []

for metalearner in metalearner_algorithm:
    print("\n\n>>>>> ", metalearner, " <<<<<<")

    ensemble = H2OStackedEnsembleEstimator(
                                       base_models= collection_of_models,

                                       model_id= "ensemble_" + metalearner

                                       #metalearner_params
                                       )
    ensemble.train(x=x, y=y, training_frame=train, validation_frame=test)
    

    ensemble_list.append(ensemble)



ensemble_list





>>>>>  auto  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%


>>>>>  xgboost  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%


>>>>>  drf  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%


>>>>>  gbm  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%


>>>>>  glm  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%


>>>>>  naivebayes  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%


>>>>>  deeplearning  <<<<<<
stackedensemble Model Build progress: |███████████████████████████████████| 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_auto

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.22624419263115159
RMSE: 0.47565133515123403
LogLoss: 0.644852966

,,0,1,Error,Rate
0,0,0.0,423.0,1.0,(423.0/423.0)
1,1,0.0,800.0,0.0,(0.0/800.0)
2,Total,0.0,1223.0,0.3459,(423.0/1223.0)


,metric,threshold,value,idx
0,max f1,0.654129,0.790905,0.0
1,max f2,0.654129,0.904364,0.0
2,max f0point5,0.654129,0.702741,0.0
3,max accuracy,0.654129,0.654129,0.0
4,max precision,0.654129,0.654129,0.0
5,max recall,0.654129,1.000000,0.0
6,max specificity,0.654129,0.000000,0.0
7,max absolute_mcc,0.654129,0.000000,0.0
8,max min_per_class_accuracy,0.654129,0.000000,0.0
9,max mean_per_class_accuracy,0.654129,0.500000,0.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,1.0,0.654129,1.0,1.0,0.654129,0.654129,0.654129,0.654129,1.0,1.0,0.0,0.0


Gini: 0.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6541291905151275: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 65.41 %

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  ensemble_xgboost

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.22624419263115159
RMSE: 0.47565133515123403
LogLoss: 0.6448529665367898
Null degrees of freedom: 1222
Residual degrees of freedom: 1222
Null deviance: 1577.310356148988
Residual deviance: 1577.310356148988
AIC: 1579.310356148988
AUC: 0.5
AUCPR: 0.6541291905151267
Gini: 0.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6541291905151275: 

...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,1.0,0.654129,1.0,1.0,0.654129,0.654129,0.654129,0.654129,1.0,1.0,0.0,0.0


,,0,1,Error,Rate
0,0,0.0,163.0,1.0,(163.0/163.0)
1,1,0.0,338.0,0.0,(0.0/338.0)
2,Total,0.0,501.0,0.3253,(163.0/501.0)


,metric,threshold,value,idx
0,max f1,0.654129,0.805721,0.0
1,max f2,0.654129,0.912035,0.0
2,max f0point5,0.654129,0.721605,0.0
3,max accuracy,0.654129,0.674651,0.0
4,max precision,0.654129,0.674651,0.0
5,max recall,0.654129,1.000000,0.0
6,max specificity,0.654129,0.000000,0.0
7,max absolute_mcc,0.654129,0.000000,0.0
8,max min_per_class_accuracy,0.654129,0.000000,0.0
9,max mean_per_class_accuracy,0.654129,0.500000,0.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,1.0,0.654129,1.0,1.0,0.674651,0.654129,0.674651,0.654129,1.0,1.0,0.0,0.0


[, , , , , , ]

In [64]:
for model in ensemble_list:
    print("AUC on test data: ",  model.model_performance(test).auc())

    

AUC on test data:  0.5
AUC on test data:  0.5
AUC on test data:  0.5
AUC on test data:  0.5
AUC on test data:  0.5
AUC on test data:  0.5
AUC on test data:  0.5


In [38]:
ensemble.model_performance

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_model_python_1604225741410_5364

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.22750895562837425
RMSE: 0.47697898866551164
LogLoss: 0.6483031547241926
Null degrees of freedom: 1222
Residual degrees of freedom: 1220
Null deviance: 1577.310356148988
Residual deviance: 1585.7495164553748
AIC: 1591.7495164553748
AUC: 0.493600768321513
AUCPR: 0.6416148141617539
Gini: -0.012798463356973988

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6532691541252658: 


,,0,1,Error,Rate
0,0,0.0,423.0,1.0,(423.0/423.0)
1,1,0.0,800.0,0.0,(0.0/800.0)
2,Total,0.0,1223.0,0.3459,(423.0/1223.0)



Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 65.41 %, avg score: 65.47 %


ModelMetricsBinomialGLM: stackedensemble
** Reported on validation data. **

MSE: 0.21956627463493159
RMSE: 0.46857899508506734
LogLoss: 0.6308890058865814
Null degrees of freedom: 500
Residual degrees of freedom: 498
Null deviance: 633.0394016859412
Residual deviance: 632.150783898354
AIC: 638.150783898354
AUC: 0.5043289650415653
AUCPR: 0.6835897020996019
Gini: 0.00865793008313065

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6532691541252656: 

Maximum Metrics: Maximum metrics at their respective thresholds

Gains/Lift Table: Avg response rate: 67.47 %, avg score: 65.54 %



,metric,threshold,value,idx
0,max f1,0.653269,0.790905,1.0
1,max f2,0.653269,0.904364,1.0
2,max f0point5,0.653269,0.702741,1.0
3,max accuracy,0.653269,0.654129,1.0
4,max precision,0.653269,0.654129,1.0
5,max recall,0.653269,1.000000,1.0
6,max specificity,0.828573,0.983452,0.0
7,max absolute_mcc,0.828573,0.067599,0.0
8,max min_per_class_accuracy,0.828573,0.003750,0.0
9,max mean_per_class_accuracy,0.653269,0.500000,1.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,1.0,0.653269,1.0,1.0,0.654129,0.654703,0.654129,0.654703,1.0,1.0,0.0,0.0


,,0,1,Error,Rate
0,0,0.0,163.0,1.0,(163.0/163.0)
1,1,0.0,338.0,0.0,(0.0/338.0)
2,Total,0.0,501.0,0.3253,(163.0/501.0)


,metric,threshold,value,idx
0,max f1,0.653269,0.805721,1.0
1,max f2,0.653269,0.912035,1.0
2,max f0point5,0.653269,0.721605,1.0
3,max accuracy,0.653269,0.674651,1.0
4,max precision,0.828573,0.833333,0.0
5,max recall,0.653269,1.000000,1.0
6,max specificity,0.828573,0.993865,0.0
7,max absolute_mcc,0.828573,0.037290,0.0
8,max min_per_class_accuracy,0.828573,0.014793,0.0
9,max mean_per_class_accuracy,0.828573,0.504329,0.0


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.011976,0.828573,1.235207,1.235207,0.833333,0.828573,0.833333,0.828573,0.014793,0.014793,23.52071,23.52071
1,,2,1.000000,0.653269,0.997149,1.000000,0.672727,0.653269,0.674651,0.655369,0.985207,1.000000,-0.28510,0.00000


<bound method ModelBase.model_performance of >